# Datos Satelitales Disponibles en Google Earth Engine (GEE)

En este proyecto usaremos diferentes variables satelitales que afectan la agricultura y pueden ser integradas al videojuego.

| Indicador / Variable            | Dataset en GEE                                   | Resolución | Qué mide (unidad) | Uso en Agricultura / Juego |
|---------------------------------|--------------------------------------------------|------------|-------------------|-----------------------------|
| **NDVI** (vegetación)           | `COPERNICUS/S2_SR_HARMONIZED` (Sentinel-2)       | 10 m       | Índice de vegetación (adimensional, -1 a 1) | Salud del cultivo, estrés |
| **EVI** (vegetación mejorada)   | `MODIS/061/MOD13Q1`                              | 250 m      | Índice de vegetación ajustado (adimensional, -1 a 1) | Más robusto en zonas densas |
| **Anomalías NDVI históricas**   | MODIS/Sentinel (comparando con medias históricas) | 250 m–10 m | Diferencia NDVI actual - promedio histórico | Detectar bajas campañas |
| **Temperatura superficial**     | `MODIS/061/MOD11A2`                              | 1 km       | Temperatura de la superficie (°C) | Estrés térmico, olas de calor |
| **Temperatura mínima / máxima** | `ECMWF/ERA5_LAND/HOURLY`                         | 9 km       | Temperatura del aire 2m (°C) | Heladas y calor extremo |
| **Precipitación diaria**        | `UCSB-CHG/CHIRPS/DAILY`                          | 5 km       | Lluvia acumulada diaria (mm/día) | Estimar agua disponible |
| **Anomalías de precipitación**  | CHIRPS histórico (comparando series temporales)  | 5 km       | Diferencia lluvia actual - promedio histórico (mm) | Fenómeno El Niño / La Niña |
| **Humedad del suelo**           | `NASA_USDA/HSL/SMAP10KM_soil_moisture`           | 10 km      | Humedad volumétrica del suelo (m³/m³) | Disponibilidad real de agua |
| **Evapotranspiración (ET)**     | `MODIS/061/MOD16A2`                              | 500 m      | Agua transferida suelo–atmósfera (mm/8 días) | Demanda hídrica del cultivo |
| **Índice de sequía (NDWI, ESI)**| MODIS/Sentinel                                   | 10–500 m   | Balance agua en vegetación (adimensional) | Estrés hídrico en plantas |
| **Biomasa (LAI, FAPAR)**        | `MODIS/061/MCD15A3H` (LAI/FAPAR)                 | 500 m      | LAI = área foliar (m²/m²), FAPAR = fracción de radiación absorbida (%) | Productividad fotosintética |
| **Cobertura terrestre**         | `MODIS/061/MCD12Q1`                              | 500 m      | Clasificación de uso del suelo (categorías) | Identificar uso agrícola del suelo |
| **Incendios / focos de calor**  | `FIRMS` (`FIRMS`)                                | 375 m      | Radiación térmica de incendios (MW) | Impacto en parcelas agrícolas |
| **Nubes persistentes**          | `COPERNICUS/S2_CLOUD_PROBABILITY`                | 10 m       | Probabilidad de nubosidad (%) | Interferencia en fotosíntesis |
| **Aerosoles / Contaminación**   | `COPERNICUS/S5P/OFFL/L3_AER_AI`                  | 1 km       | Profundidad óptica de aerosoles (adimensional) | Estrés ambiental en cultivos |

---


# 🌾 Extracción de Datos Agrícolas para FarmGuardians
## Región: Global

Este notebook extrae datos satelitales de:

- 🌱 **NDVI (Índice de Vegetación)**
- 🌡️ **Temperatura Superficial Día (LST_Day_C)**
- 🌙 **Temperatura Superficial Noche (LST_Night_C)**
- 🌧️ **Precipitación acumulada (mm / 16 días)**
- 💦 **Humedad del suelo (m³/m³)**
- ☀️ **Evapotranspiración (m)**
- 🔆 **Radiación solar (J/m²)**

**Fuente de datos:** Google Earth Engine  
(Sentinel-2, MODIS, CHIRPS, ERA5-Land)

---

## Paso 1: Clonar repositorio e instalar dependencias

En cada sesión de Colab debemos:

1. Clonar el repositorio desde GitHub  
2. Instalar las librerías necesarias desde `requirements.txt`


In [1]:
# Clonar el repositorio desde GitHub
!git clone https://github.com/iAmMazapan/FarmGuardians.git

# Instalar las dependencias desde requirements.txt
!pip install -r FarmGuardians/DataIntegrator/requirements.txt


Cloning into 'FarmGuardians'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 148 (delta 32), reused 79 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 233.49 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (32/32), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.6 MB/s eta 0:00:00


---

## Paso 2: Configurar accesos

En este paso vamos a:
1. **Montar Google Drive** - Para guardar los resultados
2. **Autenticar Google Earth Engine** - Para acceder a datos satelitales

### ¿Por qué necesitamos esto?

- **Google Drive**: Los datos extraídos (CSV, JSON) se guardarán aquí para no perderlos
- **Google Earth Engine**: Es la plataforma de Google que nos da acceso gratuito a petabytes de imágenes satelitales

### Notas importantes:
- Te pedirá permisos para acceder a tu Drive
- La autenticación de GEE es mediante una cuenta de Google
- Solo necesitas hacerlo una vez por sesión

In [2]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive/')

print("✅ Google Drive montado en /content/drive/")
print("-" * 50)

# 2. Autenticar Google Earth Engine
import ee

print("\n Iniciando autenticación de Google Earth Engine...")
print("Se abrirá una ventana para autenticarte con tu cuenta de Google")

# Autenticar (se abre ventana emergente)
ee.Authenticate()

# Inicializar (sin proyecto específico por ahora)
ee.Initialize(project='farmguardians-gee')

print(" Google Earth Engine autenticado correctamente")
print("-" * 50)
print("\n🎉 Todo listo para extraer datos satelitales!")

Mounted at /content/drive/
✅ Google Drive montado en /content/drive/
--------------------------------------------------

 Iniciando autenticación de Google Earth Engine...
Se abrirá una ventana para autenticarte con tu cuenta de Google
 Google Earth Engine autenticado correctamente
--------------------------------------------------

🎉 Todo listo para extraer datos satelitales!


---

## Paso 4: Definir ubicaciones y fenómenos de estudio

En esta sección definimos una lista de **ubicaciones agrícolas** en distintas partes del mundo donde ocurrieron fenómenos climáticos o ambientales relevantes.  

Cada ubicación incluye:  
- 🌍 **País / región**  
- 📍 **Ubicación y coordenadas**  
- 📅 **Periodo de estudio (fecha de inicio y fin)**  
- 🌱 **Descripción del fenómeno y su impacto en la agricultura**  

Esto nos permitirá extraer y analizar las variables satelitales (NDVI, temperatura superficial, precipitación, humedad del suelo, etc.) en el contexto de eventos reales que afectaron a la agricultura.

**Fenómenos considerados:**
1. **Sequía extrema** en California (EE. UU., 2014-2015)  
2. **Calor extremo** en Australia (2019-2020)  
3. **Plagas agrícolas** en Punjab, India (2020)  
4. **Precipitaciones fluviales** en Bangladesh (2017)  
5. **Heladas / frío extremo** en Siberia, Rusia (2018)  
6. **Fenómeno de El Niño / lluvias intensas** en Perú (2017)  

El código siguiente define esta información en una lista de diccionarios de Python llamada `ubicaciones_fenomenos`.


In [3]:
ubicaciones_fenomenos = [
    {
        "fenomeno": "Sequía extrema",
        "pais_region": "California, EE. UU.",
        "ubicacion": "Valle Central / zona agrícola cerca de Fresno",
        "coordenadas": [-119.7871, 36.7378],
        "fecha_inicio": "2014-07-01",
        "fecha_fin": "2015-06-30",
        "descripcion": "Sequía prolongada afectando agricultura e incendios forestales"
    },
    {
        "fenomeno": "Calor extremo",
        "pais_region": "Australia (Sur y Este)",
        "ubicacion": "zona agrícola en Victoria cerca de Melbourne",
        "coordenadas": [145.0, -37.8],
        "fecha_inicio": "2019-03-01",
        "fecha_fin": "2020-02-29",
        "descripcion": "Ola de calor récord afectando estrés en cultivos"
    },
    {
        "fenomeno": "Plagas agrícolas",
        "pais_region": "India (Punjab)",
        "ubicacion": "zona agrícola en Punjab cerca de Ludhiana",
        "coordenadas": [75.85, 30.90],
        "fecha_inicio": "2019-12-01",
        "fecha_fin": "2020-11-30",
        "descripcion": "Brotes de langostas del desierto afectando trigo y otros cultivos"
    },
    {
        "fenomeno": "Precipitaciones fluviales",
        "pais_region": "Bangladesh (Delta del Ganges)",
        "ubicacion": "zona agrícola en los distritos inundables cerca de Dhaka",
        "coordenadas": [90.5, 23.8],
        "fecha_inicio": "2016-12-01",
        "fecha_fin": "2017-11-30",
        "descripcion": "Inundaciones por lluvias monzónicas, afectando arrozales"
    },
    {
        "fenomeno": "Heladas / Frío extremo",
        "pais_region": "Rusia (Siberia)",
        "ubicacion": "zona agrícola en Siberia central dentro del polígono",
        "coordenadas": [105.5, 61.8],
        "fecha_inicio": "2017-07-01",
        "fecha_fin": "2018-06-30",
        "descripcion": "Heladas severas que afectaron cultivos de cereales y hortalizas"
    },
    {
        "fenomeno": "Fenómeno de El Niño / Lluvias intensas",
        "pais_region": "Perú (Costa norte y central)",
        "ubicacion": "zona agrícola en Piura / costa norte",
        "coordenadas": [-80.63, -5.19],
        "fecha_inicio": "2016-07-01",
        "fecha_fin": "2017-06-30",
        "descripcion": "Lluvias extremas, inundaciones y desbordes de ríos, afectando cultivos y suelos"
    }
]


In [13]:
import ee
import geemap
import pandas as pd

# --- Datos: tu lista de diccionarios ---
# Asegúrate de tener algo así definido:
# ubicaciones_fenomenos = [
#     {
#         "fenomeno": "Sequía extrema",
#         "pais_region": "California, EE. UU.",
#         "ubicacion": "Valle Central / Fresno",
#         "coordenadas": [-119.7871, 36.7378],
#         "fecha_inicio": "2025-01-01",
#         "fecha_fin": "2025-01-16",
#         "descripcion": "Zona agrícola con sequía prolongada"
#     },
#     ...
# ]

# Convertir la lista de diccionarios en DataFrame
df = pd.DataFrame(ubicaciones_fenomenos)
df["lon"] = df["coordenadas"].apply(lambda x: x[0])
df["lat"] = df["coordenadas"].apply(lambda x: x[1])

# Crear mapa global centrado
mapa_global = geemap.Map(center=[10, 0], zoom=2)
mapa_global.add_basemap("HYBRID")

# Añadir puntos desde el DataFrame
mapa_global.add_points_from_xy(
    df,
    x="lon",
    y="lat",
    popup=["fenomeno", "ubicacion", "pais_region", "fecha_inicio", "fecha_fin", "descripcion"],
    color="red",
    marker_type="circle",
    point_size=10
)

# Guardar el mapa como imagen estática
# Guardar el mapa como imagen estática (sin width/height)
mapa_global.to_html("mapa_fenomenos.html")


print("Mapa guardado como 'mapa_fenomenos.png', listo para subir a GitHub.")


Mapa guardado como 'mapa_fenomenos.png', listo para subir a GitHub.


---

## Paso 4: Definir funciones de extracción y procesamiento de datos

En este paso creamos las funciones que permiten extraer y procesar datos climáticos y ambientales desde Google Earth Engine (GEE), incluyendo NDVI, temperatura, precipitación, humedad del suelo, evapotranspiración, radiación solar y días de helada.

### Funciones principales

1. **`extraer_datos_originales(lugar)`**  
   Extrae los datos en su frecuencia nativa (ej. NDVI cada 16 días).  
   - Crea un buffer de 1 km alrededor de las coordenadas para capturar un área representativa.  
   - Extrae:
     - **NDVI**: de MODIS (`MOD13Q1`) escalado a valores 0–1.  
     - **Temperatura superficial (LST Day/Night)**: de MODIS (`MOD11A1`), convertida de Kelvin a °C.  
     - **Precipitación diaria**: de CHIRPS, acumulada sobre ventanas de 16 días.  
     - **ERA5-Land**: humedad del suelo, evapotranspiración y radiación solar.  
   - Calcula los **días de helada** cuando la temperatura nocturna es < 0°C.  
   - Devuelve un DataFrame con todos los valores originales.

2. **`interpolar_cada_3_dias(df_original, fecha_inicio, fecha_fin)`**  
   - Genera una serie temporal con un intervalo de 3 días.  
   - Interpola linealmente los valores faltantes para las variables continuas (NDVI, temperatura, precipitación, etc.).  
   - Propaga los valores de días de helada con forward-fill (0 si faltan).  
   - Añade una columna indicadora `dato_original` para diferenciar datos reales de interpolados.  

3. **`exportar_archivos(df_interpolado, df_original, lugar, nombre_archivo, carpeta_salida)`**  
   - Exporta:
     - CSV de datos interpolados cada 3 días.  
     - CSV de datos originales en frecuencia nativa.  
     - JSON con metadatos completos, estadísticas y todos los datos.  

4. **`procesar_todas_ubicaciones(ubicaciones_fenomenos, exportar=True, carpeta_salida='datos_climaticos')`**  
   - Itera sobre todas las ubicaciones definidas y aplica las funciones anteriores.  
   - Devuelve un diccionario con los DataFrames originales e interpolados de cada lugar.  
   - Exporta automáticamente los archivos CSV y JSON si `exportar=True`.  

### Por qué tantas funciones

Cada conjunto de datos tiene características específicas y requiere un procesamiento distinto:

- **MODIS NDVI**: ya calculado, necesita escalar.  
- **MODIS LST**: escala y conversión de Kelvin a Celsius.  
- **CHIRPS**: precipitación lista para usar, pero acumulada sobre la ventana de tiempo.  
- **ERA5-Land**: variables derivadas de modelos (humedad, evapotranspiración, radiación).  
- **Interpolación**: garantiza una serie temporal uniforme cada 3 días para análisis posteriores.  


In [5]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Función para extraer datos originales

In [6]:
def extraer_datos_originales(lugar):
    """
    Extrae datos en sus frecuencias nativas (principalmente cada 16 días para NDVI)
    """
    geom = ee.Geometry.Point(lugar["coordenadas"])
    start = lugar["fecha_inicio"]
    end = lugar["fecha_fin"]

    # Buffer de 1km para capturar área representativa
    region = geom.buffer(1000)

    print(f"\n{'='*60}")
    print(f"Procesando: {lugar['ubicacion']} ({lugar['pais_region']})")
    print(f"Fenómeno: {lugar['fenomeno']}")
    print(f"Período: {start} a {end}")
    print(f"{'='*60}")

    # Colecciones de datos
    ndvi_coll = ee.ImageCollection('MODIS/006/MOD13Q1').filterDate(start, end).filterBounds(geom).select('NDVI')
    lst_coll = ee.ImageCollection('MODIS/006/MOD11A1').filterDate(start, end).filterBounds(geom).select(['LST_Day_1km', 'LST_Night_1km'])
    precip_coll = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(start, end).filterBounds(geom).select('precipitation')
    era5_coll = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).filterBounds(geom)

    # Verificar disponibilidad de datos
    ndvi_size = ndvi_coll.size().getInfo()
    if ndvi_size == 0:
        print(f"  No hay datos NDVI disponibles para este rango de fechas")
        return pd.DataFrame()

    # Obtener fechas disponibles (NDVI cada 16 días - será nuestra base)
    fechas_ms = ndvi_coll.aggregate_array('system:time_start').getInfo()
    print(f"  {len(fechas_ms)} imágenes NDVI encontradas")

    datos = []
    errores = 0

    for i, fecha_ms in enumerate(fechas_ms):
        try:
            fecha = pd.to_datetime(fecha_ms, unit='ms')
            fecha_str = fecha.strftime('%Y-%m-%d')

            # Ventanas de tiempo
            fecha_inicio_ventana = ee.Date(fecha_ms)
            fecha_fin_ventana = fecha_inicio_ventana.advance(1, 'day')

            # NDVI (escalar por 0.0001)
            ndvi_img = ndvi_coll.filterDate(fecha_inicio_ventana, fecha_fin_ventana).first()
            ndvi_val = ndvi_img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=250,
                crs='EPSG:4326',
                maxPixels=1e9
            ).get('NDVI').getInfo()
            ndvi_val = ndvi_val * 0.0001 if ndvi_val is not None else None

            # LST (escalar por 0.02 y convertir a Celsius)
            lst_img = lst_coll.filterDate(
                fecha_inicio_ventana.advance(-1, 'day'),
                fecha_fin_ventana.advance(1, 'day')
            ).first()

            if lst_img:
                lst_dict = lst_img.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=region,
                    scale=1000,
                    crs='EPSG:4326',
                    maxPixels=1e9
                ).getInfo()
                lst_day = (lst_dict.get('LST_Day_1km') * 0.02 - 273.15) if lst_dict.get('LST_Day_1km') else None
                lst_night = (lst_dict.get('LST_Night_1km') * 0.02 - 273.15) if lst_dict.get('LST_Night_1km') else None
            else:
                lst_day, lst_night = None, None

            # Precipitación (acumulada en ventana de 16 días)
            precip_imgs = precip_coll.filterDate(
                fecha_inicio_ventana.advance(-8, 'day'),
                fecha_fin_ventana.advance(8, 'day')
            )

            if precip_imgs.size().getInfo() > 0:
                precip_sum = precip_imgs.sum()
                precip_val = precip_sum.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=region,
                    scale=5000,
                    crs='EPSG:4326',
                    maxPixels=1e9
                ).get('precipitation').getInfo()
            else:
                precip_val = None

            # ERA5-Land (promedio del día)
            era5_day = era5_coll.filterDate(
                fecha_inicio_ventana.advance(-12, 'hour'),
                fecha_fin_ventana.advance(12, 'hour')
            )

            if era5_day.size().getInfo() > 0:
                era5_mean = era5_day.mean()
                era5_dict = era5_mean.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=region,
                    scale=11132,
                    crs='EPSG:4326',
                    maxPixels=1e9
                ).getInfo()

                soil_moist = era5_dict.get('volumetric_soil_water_layer_1')
                evapotransp = era5_dict.get('evaporation_from_vegetation_transpiration')
                rad_solar = era5_dict.get('surface_solar_radiation_downwards')
            else:
                soil_moist, evapotransp, rad_solar = None, None, None

            # Días de helada (LST nocturno < 0°C)
            if lst_night is not None and lst_night < 0:
                dias_helada = 1
            else:
                dias_helada = 0

            datos.append({
                "fecha": fecha_str,
                "NDVI": round(ndvi_val, 4) if ndvi_val is not None else None,
                "LST_Day_C": round(lst_day, 2) if lst_day is not None else None,
                "LST_Night_C": round(lst_night, 2) if lst_night is not None else None,
                "Precipitacion_mm_16dias": round(precip_val, 2) if precip_val is not None else None,
                "Humedad_suelo_m3m3": round(soil_moist, 4) if soil_moist is not None else None,
                "Evapotranspiracion_m": round(evapotransp, 6) if evapotransp is not None else None,
                "Radiacion_solar_Jm2": round(rad_solar, 2) if rad_solar is not None else None,
                "Dias_helada": dias_helada
            })

            # Mostrar progreso
            if (i + 1) % 10 == 0 or (i + 1) == len(fechas_ms):
                print(f"  Progreso: {i+1}/{len(fechas_ms)} imágenes procesadas")

        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"  Error en fecha {fecha_str}: {str(e)[:100]}")
            continue

    if errores > 0:
        print(f"  Total de errores: {errores}/{len(fechas_ms)}")

    df = pd.DataFrame(datos)
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df.sort_values('fecha').reset_index(drop=True)

    print(f"  Datos originales extraídos: {len(df)} registros")

    return df

## Función de interpolación

In [7]:
def interpolar_cada_3_dias(df_original, fecha_inicio, fecha_fin):
    """
    Crea una serie temporal cada 3 días e interpola linealmente los valores faltantes
    """
    if df_original.empty:
        return pd.DataFrame()

    print(f"\n  Interpolando datos cada 3 días...")

    # Crear rango de fechas cada 3 días
    fecha_inicio_dt = pd.to_datetime(fecha_inicio)
    fecha_fin_dt = pd.to_datetime(fecha_fin)

    fechas_3dias = pd.date_range(start=fecha_inicio_dt, end=fecha_fin_dt, freq='3D')

    # Crear DataFrame con todas las fechas cada 3 días
    df_interpolado = pd.DataFrame({'fecha': fechas_3dias})

    # Hacer merge con los datos originales
    df_merged = df_interpolado.merge(df_original, on='fecha', how='left')

    # Columnas numéricas para interpolar (excluir Dias_helada ya que es binario)
    columnas_interpolar = [
        'NDVI', 'LST_Day_C', 'LST_Night_C', 'Precipitacion_mm_16dias',
        'Humedad_suelo_m3m3', 'Evapotranspiracion_m', 'Radiacion_solar_Jm2'
    ]

    # Interpolar linealmente cada columna
    for col in columnas_interpolar:
        if col in df_merged.columns:
            df_merged[col] = df_merged[col].interpolate(method='linear', limit_area='inside')
            # Redondear según precisión original
            if col == 'NDVI':
                df_merged[col] = df_merged[col].round(4)
            elif col in ['Humedad_suelo_m3m3']:
                df_merged[col] = df_merged[col].round(4)
            elif col in ['Evapotranspiracion_m']:
                df_merged[col] = df_merged[col].round(6)
            else:
                df_merged[col] = df_merged[col].round(2)

    # Para Dias_helada, usar forward fill (propagar el valor anterior)
    if 'Dias_helada' in df_merged.columns:
        df_merged['Dias_helada'] = df_merged['Dias_helada'].fillna(0).astype(int)

    # Añadir columna indicadora de si el dato es original o interpolado
    df_merged['dato_original'] = df_merged['NDVI'].notna() & (df_merged['fecha'].isin(df_original['fecha']))

    print(f"  Serie temporal creada: {len(df_merged)} registros cada 3 días")
    print(f"  Datos originales: {df_merged['dato_original'].sum()}")
    print(f"  Datos interpolados: {(~df_merged['dato_original']).sum()}")

    return df_merged


## Función de exportación

In [8]:
def exportar_archivos(df_interpolado, df_original, lugar, nombre_archivo, carpeta_salida):
    """
    Exporta los datos a CSV y JSON con metadatos
    """
    import json

    print(f"\n  Exportando archivos...")

    # EXPORTAR CSV (datos interpolados cada 3 días)
    csv_path = f"{carpeta_salida}/{nombre_archivo}_datos_3dias.csv"
    df_interpolado_export = df_interpolado.copy()
    df_interpolado_export['fecha'] = df_interpolado_export['fecha'].dt.strftime('%Y-%m-%d')
    df_interpolado_export.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"    CSV guardado: {csv_path}")

    # EXPORTAR CSV (datos originales)
    csv_original_path = f"{carpeta_salida}/{nombre_archivo}_datos_originales.csv"
    df_original_export = df_original.copy()
    df_original_export['fecha'] = df_original_export['fecha'].dt.strftime('%Y-%m-%d')
    df_original_export.to_csv(csv_original_path, index=False, encoding='utf-8')
    print(f"    CSV original guardado: {csv_original_path}")

    # EXPORTAR JSON con metadatos
    json_path = f"{carpeta_salida}/{nombre_archivo}_completo.json"

    # Preparar datos para JSON
    datos_interpolados_json = df_interpolado_export.to_dict(orient='records')
    datos_originales_json = df_original_export.to_dict(orient='records')

    # Calcular estadísticas
    estadisticas = {
        "total_registros_interpolados": len(df_interpolado),
        "total_registros_originales": len(df_original),
        "registros_interpolados": int((~df_interpolado['dato_original']).sum()),
        "fecha_inicio": lugar['fecha_inicio'],
        "fecha_fin": lugar['fecha_fin'],
        "variables_disponibles": list(df_interpolado.columns.drop(['fecha', 'dato_original'])),
        "datos_faltantes": {
            col: int(df_interpolado[col].isna().sum())
            for col in df_interpolado.columns
            if col not in ['fecha', 'dato_original']
        }
    }

    # Estructura completa del JSON
    datos_json = {
        "metadata": {
            "ubicacion": lugar['ubicacion'],
            "pais_region": lugar['pais_region'],
            "coordenadas": {
                "longitud": lugar['coordenadas'][0],
                "latitud": lugar['coordenadas'][1]
            },
            "fenomeno": lugar['fenomeno'],
            "descripcion": lugar['descripcion'],
            "periodo": {
                "inicio": lugar['fecha_inicio'],
                "fin": lugar['fecha_fin']
            },
            "frecuencia_datos": "cada_3_dias",
            "metodo_interpolacion": "lineal"
        },
        "estadisticas": estadisticas,
        "datos_interpolados_3dias": datos_interpolados_json,
        "datos_originales_nativos": datos_originales_json
    }

    # Guardar JSON con formato legible
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(datos_json, f, ensure_ascii=False, indent=2)

    print(f"    JSON guardado: {json_path}")

    # RESUMEN DE ARCHIVOS
    print(f"\n  Archivos generados para {nombre_archivo}:")
    print(f"    1. {nombre_archivo}_datos_3dias.csv - Datos interpolados cada 3 días")
    print(f"    2. {nombre_archivo}_datos_originales.csv - Datos en frecuencia nativa")
    print(f"    3. {nombre_archivo}_completo.json - Datos + metadatos completos")


## Funcion principal de procesamiento

In [9]:
def procesar_todas_ubicaciones(ubicaciones_fenomenos, exportar=True, carpeta_salida='datos_climaticos'):
    """
    Procesa todas las ubicaciones y retorna un diccionario con los DataFrames
    Opcionalmente exporta los datos a CSV y JSON
    """
    import os
    import json

    resultados = {}

    # Crear carpeta de salida si se va a exportar
    if exportar:
        os.makedirs(carpeta_salida, exist_ok=True)
        print(f"\nCarpeta de salida: {carpeta_salida}/")

    print("\n" + "="*60)
    print("INICIANDO EXTRACCIÓN DE DATOS CLIMÁTICOS")
    print("="*60)

    for i, lugar in enumerate(ubicaciones_fenomenos, 1):
        print(f"\n\n[{i}/{len(ubicaciones_fenomenos)}] Procesando ubicación...")

        try:
            # Extraer datos originales
            df_original = extraer_datos_originales(lugar)

            if not df_original.empty:
                # Interpolar cada 3 días
                df_interpolado = interpolar_cada_3_dias(
                    df_original,
                    lugar['fecha_inicio'],
                    lugar['fecha_fin']
                )

                # Crear nombre de archivo limpio
                ubicacion_limpia = lugar['ubicacion'].replace('/', '_').replace(' ', '_')
                pais_limpio = lugar['pais_region'].split(',')[0].strip().replace(' ', '_')
                nombre_archivo = f"{ubicacion_limpia}_{pais_limpio}"

                # Guardar ambos DataFrames
                key = nombre_archivo
                resultados[key] = {
                    'info': lugar,
                    'datos_originales': df_original,
                    'datos_interpolados': df_interpolado
                }

                # Exportar archivos si está habilitado
                if exportar:
                    exportar_archivos(df_interpolado, df_original, lugar, nombre_archivo, carpeta_salida)

                print(f"  Ubicación procesada exitosamente")
            else:
                print(f"  No se pudieron extraer datos para esta ubicación")

        except Exception as e:
            print(f"  Error general en ubicación: {str(e)}")
            continue

    print("\n" + "="*60)
    print(f"PROCESO COMPLETADO: {len(resultados)} ubicaciones procesadas")
    print("="*60)

    return resultados


## Ejecucion Final

In [10]:

#EJECUCION FINAL DEL PROCESAMIENTO

# Procesar todas las ubicaciones CON EXPORTACIÓN AUTOMÁTICA
resultados = procesar_todas_ubicaciones(
    ubicaciones_fenomenos,
    exportar=True,
    carpeta_salida='datos_climaticos'
)


# ACCEDER A LOS RESULTADOS

print("\nUbicaciones procesadas:")
for key in resultados.keys():
    print(f"  - {key}")

print(f"\nTodos los archivos CSV y JSON han sido generados")
print(f"Total: {len(resultados)} ubicaciones x 3 archivos = {len(resultados)*3} archivos")


Carpeta de salida: datos_climaticos/

INICIANDO EXTRACCIÓN DE DATOS CLIMÁTICOS


[1/6] Procesando ubicación...

Procesando: Valle Central / zona agrícola cerca de Fresno (California, EE. UU.)
Fenómeno: Sequía extrema
Período: 2014-07-01 a 2015-06-30
  23 imágenes NDVI encontradas
  Progreso: 10/23 imágenes procesadas
  Progreso: 20/23 imágenes procesadas
  Progreso: 23/23 imágenes procesadas
  Datos originales extraídos: 23 registros

  Interpolando datos cada 3 días...
  Serie temporal creada: 122 registros cada 3 días
  Datos originales: 8
  Datos interpolados: 114

  Exportando archivos...
    CSV guardado: datos_climaticos/Valle_Central___zona_agrícola_cerca_de_Fresno_California_datos_3dias.csv
    CSV original guardado: datos_climaticos/Valle_Central___zona_agrícola_cerca_de_Fresno_California_datos_originales.csv
    JSON guardado: datos_climaticos/Valle_Central___zona_agrícola_cerca_de_Fresno_California_completo.json

  Archivos generados para Valle_Central___zona_agrícola_cerca

## Mover archivos a github